# 📊 Prédiction des prix des voitures 

L'objectif de ce projet est de réaliser une étude sur le marché des voitures d'occasions afin de déterminer quelles voitures ont des prix inférieurs selon les caractéristiques et de pouvoir réaliser de bonnes affaires sur le marché.

Pour ce faire, nous avons réalisé un web scraping du site https://www.autoscout24.be/fr/ nous permettant de récupérer et d'avoir une base de données en .csv pour le machine learning.

Nous avons nettoyer la base de données puis réaliser une analyse descriptive pour mieux comprendre les différentes variables.

Enfin les données ont été entrainé sur les modèles de MACHINE LEARNING afin de ressortir un modèle les voitures ayant un prix inférieur en dessous du prix prédit pour leurs caractéristiques.

Scraping
Pour le scraping, nous avons utilisé trois (3) fichiers, le fichier scraper_methods.py pour obtenir les contenus des pages HTML, puis le fichier VehicleScraper.py avec la classe VehicleScraper pour extraction des caractéristiques de voitures de chaque annonce et à la fin main.py utilisant VehicleScraper pour le scraping de plusieurs pages et enregistrer sous vehicule.csv.

Les librairies

In [2]:
import os
import pandas as pd
import numpy as np  
import seaborn as sns 
import plotly.express as px
import matplotlib.pyplot as plt

from pathlib import Path 

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
from scipy.optimize import minimize_scalar
import joblib


sns.set_theme(style="whitegrid", palette="muted")
from matplotlib import rcParams
palette = sns.color_palette("pastel")

import warnings
warnings.filterwarnings("ignore")

**Les données**  

In [11]:
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / 'Donnees'

#print(f"BASE_DIR: {BASE_DIR}")
#print(f"DATA_DIR: {DATA_DIR}")


df = pd.read_csv(DATA_DIR / 'vehicules.csv')
df.head(4)

Modèle       Prix       Transmission  \
0   audi   € 74 999  Boîte automatique   
1   audi  € 118 900  Boîte automatique   
2   audi  € 118 900  Boîte automatique   
3   audi   € 59 995  Boîte automatique   

                                        Version Kilométrage  \
0  55 TFSI Quattro Tiptronic S-Line INDIVIDUAL!   34 000 km   
1   60 Hybride 49 CO2 BlackPack B&O 23' Leather       10 km   
2  60 Hybr 49gr Sline BlackPack B&O Leather 23'       10 km   
3               50 TDi S-Line Quattro Tiptronic   95 702 km   

            Carburant        Puissance Évaluations  \
0             Essence  250 kW (340 CH)         121   
1  Electrique/Essence  340 kW (462 CH)         107   
2  Electrique/Essence  340 kW (462 CH)         107   
3              Diesel  210 kW (286 CH)          75   

                                             Vendeur  \
0                    Kristof D'herde • BE-9300 Aalst   
1  Frederik Rik Maxime Jorn Hendrik • BE-8710 Wie...   
2  Frederik Rik Maxime Jorn Hendrik • BE-8710 Wie...   
3           Mhr Jannick Lazoore • BE-8620 Nieuwpoort   

                                   Nom de la Voiture     Date  
0  Audi Q855 TFSI Quattro Tiptronic S-Line INDIVI...  01/2021  
1  Audi Q860 Hybride 49 CO2 BlackPack B&O 23' Lea...  01/2024  
2  Audi Q860 Hybr 49gr Sline BlackPack B&O Leathe...  01/2024  
3             Audi Q850 TDi S-Line Quattro Tiptronic  04/2019

**Nettoyage et traitement des données**

In [12]:
from cleaning_data.cleaning import nettoie 

df = nettoie(df)
df.head(4)

modèle      prix       transmission  kilométrage           carburant  \
0   audi   74999.0  Boîte automatique      34000.0             Essence   
1   audi  118900.0  Boîte automatique         10.0  Electrique/Essence   
2   audi  118900.0  Boîte automatique         10.0  Electrique/Essence   
3   audi   59995.0  Boîte automatique      95702.0              Diesel   

   puissance  évaluations  
0      340.0        121.0  
1      462.0        107.0  
2      462.0        107.0  
3      286.0         75.0

In [13]:
print(f"la dimension de notre table  est : {df.shape}")

la dimension de notre table  est : (2670, 7)


In [15]:
df.describe()

prix    kilométrage    puissance  évaluations
count    2670.000000    2670.000000  2670.000000  2670.000000
mean    96578.187266   53196.666667   352.940075    71.760300
std    110014.952759   44613.449888   222.026120   101.638555
min      3900.000000       0.000000    69.000000     0.000000
25%     22000.000000   14500.000000   140.000000     9.000000
50%     54950.000000   44467.000000   286.000000    38.000000
75%    119950.000000   89088.000000   581.000000    93.000000
max    749995.000000  222222.000000   850.000000   671.000000

Variable	| Observations
prix	    | Moyenne ≈ 96 578€, max très élevé à 749 995€, présence de valeurs extrêmes, distribution probablement très asymétrique (positive).
kilométrage	| Large plage (0 à 222k km), médiane ≈ 44k, données probablement asymétriques aussi.
puissance	| Varie de 69 à 850 chevaux, ça va de citadine à supercar.
évaluations	| Distribution fortement déséquilibrée, médiane à 38 mais max à 671 ! Beaucoup de véhicules avec peu d’évaluations.

L'analyse exploratoire complète des données dans https://github.com/nazif96/Car-price/blob/main/Notebooks/EDA.ipynb.

## Machine Learning 

**Preprocessing des données**